In [ ]:
!pip install SpeechRecognition

# Install eSpeak
!sudo apt-get install espeak

# Example of using eSpeak for phoneme conversion
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 41.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,382 kB of archives.
After this operation, 3,178 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak-data amd64 1.48.15+dfsg-3 [1,085 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libespeak1 amd64 1.48.15+dfsg-3 [156 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/univ

google api  + espeak

In [ ]:
import speech_recognition as sr
import subprocess

# Initialize recognizer
recognizer = sr.Recognizer()

# Load audio file
audio_file = "/content/record_out (3).wav"

with sr.AudioFile(audio_file) as source:
    audio_data = recognizer.record(source)
    # Recognize speech using Google Web Speech API
    try:
        text = recognizer.recognize_google(audio_data)
        print("Transcribed Text: ", text)
    except sr.UnknownValueError:
        print("Google Web Speech API could not understand audio")
        text = ""
    except sr.RequestError as e:
        print(f"Could not request results from Google Web Speech API; {e}")
        text = ""

# Use espeak to convert text to IPA
if text:
    espeak_command = f'espeak --ipa "{text}"'
    process = subprocess.Popen(espeak_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    ipa_transcription = stdout.decode('utf-8').strip()
    print("IPA Transcription: ", ipa_transcription)

Transcribed Text:  Tatum
IPA Transcription:  tˈɑːtəm


wav2vec model + espeak

In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import subprocess

# Load the audio file
audio_file = "/content/p_28781562_829.wav"

# Load Wav2Vec2 processor and model
model_name = "facebook/wav2vec2-large-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Load the audio file and resample to 16kHz
waveform, sample_rate = torchaudio.load(audio_file)
waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

# Process the audio input
input_values = processor(waveform.squeeze(), return_tensors="pt", sampling_rate=16000).input_values

# Perform inference
with torch.no_grad():
    logits = model(input_values).logits

# Get the predicted IDs and decode them to text
predicted_ids = torch.argmax(logits, dim=-1)
transcribed_text = processor.batch_decode(predicted_ids)[0]

print("Transcribed Text: ", transcribed_text)

# Use espeak to convert text to IPA
if transcribed_text:
    espeak_command = f'espeak --ipa "{transcribed_text}"'
    process = subprocess.Popen(espeak_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    ipa_transcription = stdout.decode('utf-8').strip()
    print("IPA Transcription: ", ipa_transcription)


Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You s

Transcribed Text:  EXPRESSO
IPA Transcription:  ɛkspɹˈɛsəʊ
